## Análisis de Tarifas y Costos de Energía Eléctrica para el Mercado Regulado

Este conjunto de datos fue extraído del portal de **Datos Abiertos de Colombia**, y corresponde a la información de tarifas y costos de energía eléctrica para usuarios del mercado regulado, específicamente del archivo *Tarifas_y_Costos_de_Energía_Eléctrica_para_el_Mercado_Regulado_20250429*.

### Descripción del conjunto de datos

Cada fila representa el costo de prestación del servicio eléctrico a un usuario residencial o comercial, detallando distintos componentes que conforman el precio total que paga el consumidor. El dataset contiene los siguientes atributos numéricos:

- `CU Total`: Costo unitario total de la energía por kilovatio hora ($/kWh).
- `Costo Compra (Gm,i)`: Costo de compra de energía.
- `Cargo Transporte STN (Tm)`: Costo de transporte en el Sistema de Transmisión Nacional.
- `Cargo Transporte SDL (Dn,m)`: Costo de transporte en el Sistema de Distribución Local.
- `Margen Comercialización (CVm,i,j)`: Margen para cubrir costos de comercialización.
- `Costo G, T, Pérdidas (PRn,m)`: Costo total de generación, transporte y pérdidas.
- `Restricciones (Rm)`: Costo asociado a restricciones operativas en la red.
- `Cfm,j ($/fact.)`: Costo fijo mensual por factura.
- `COT`: Costo total facturado.

### Objetivo del análisis

Vamos a construir un árbol KD (k-d tree) utilizando como variables de entrada los atributos numéricos anteriores, y definiremos una **clasificación multiclase** según el valor de `CU Total`, que será nuestra **variable objetivo (target)**.

Para ello, crearemos 3 clases:

- **Clase 0**: `CU Total` bajo (menor o igual al percentil 33).
- **Clase 1**: `CU Total` medio (entre el percentil 33 y el percentil 66).
- **Clase 2**: `CU Total` alto (mayor al percentil 66).

Esta transformación convierte el problema en una tarea de clasificación supervisada basada en rangos del costo total unitario. Esto nos permitirá construir reglas de decisión y verificar el rendimiento del árbol KD en esta clasificación.


In [16]:
import pandas as pd
import numpy as np

# Cargar el dataset
df = pd.read_csv("Dataset.csv")

# Limpiar nombres de columnas (quitar espacios extra)
df.columns = df.columns.str.strip()

# Calcular percentiles para clasificar 'CU Total'
p33 = np.percentile(df["CU Total"], 33)
p66 = np.percentile(df["CU Total"], 66)

# Crear función de clasificación
def clasificar(cu):
    if cu <= p33:
        return 0
    elif cu <= p66:
        return 1
    else:
        return 2

# Aplicar clasificación
df["CU Total"] = df["CU Total"].apply(clasificar)

# Guardar nuevo dataset con clases
df.to_csv("Dataset_clases.csv", index=False)

# Mostrar primeros registros para verificar
df[["CU Total", "Costo Compra (Gm,i)",	"Cargo Transporte STN (Tm)",	"Cargo Transporte SDL (Dn,m)",	"Margen Comercialización (CVm,i,j)",	"Costo G, T, Pérdidas (PRn,m)",	"Restricciones (Rm)",	"Cfm,j ($/fact.)",	"COT"
]].head(10)


,CU Total,"Costo Compra (Gm,i)",Cargo Transporte STN (Tm),"Cargo Transporte SDL (Dn,m)","Margen Comercialización (CVm,i,j)","Costo G, T, Pérdidas (PRn,m)",Restricciones (Rm),"Cfm,j ($/fact.)",COT
0,1,289.09,54.32,256.98,70.27,58.26,18.49,9286.0,66.32
1,1,289.09,54.32,234.24,70.27,58.26,18.49,9286.0,10.83
2,1,289.09,54.32,211.50,70.27,58.26,18.49,9286.0,42.66
3,0,289.09,54.32,158.35,70.27,19.73,18.49,9286.0,83.74
4,0,289.09,54.32,110.75,70.27,19.30,18.49,9286.0,34.96
5,2,289.09,54.32,261.91,112.06,57.36,19.21,15291.0,71.35
6,2,289.09,54.32,225.50,112.06,57.36,19.21,15291.0,69.42
7,1,289.09,54.32,189.09,112.06,57.36,19.21,15291.0,76.51
8,0,289.09,54.32,157.72,112.06,24.73,19.21,15291.0,48.39
9,0,289.09,54.32,94.45,112.06,21.23,19.21,15291.0,31.05


## Construcción del Árbol KD

El Árbol KD (k-dimensional) es una estructura binaria que divide recursivamente un conjunto de datos multidimensionales utilizando ejes alternos (por ejemplo, x, luego y, luego x otra vez, etc.). En cada nodo:

1. Se selecciona un eje de división que rota entre los atributos disponibles.
2. Se calcula el umbral como la mediana del atributo en el eje seleccionado.
3. Se dividen los puntos en dos subconjuntos:
   - Lado izquierdo (valores ≤ mediana).
   - Lado derecho (valores > mediana).
4. Se repite el proceso recursivamente en cada subconjunto hasta que:
   - Quede un solo punto.
   - O se alcance una condición de parada como un número mínimo de elementos por nodo.

Este árbol nos permitirá hacer reglas de clasificación sobre el conjunto de datos y visualizar su estructura como un grafo interactivo.


In [17]:
import numpy as np

class KDNode:
    def __init__(self, point, label, axis, left=None, right=None):
        self.point = point  # coordenadas (atributos)
        self.label = label  # clase
        self.axis = axis    # eje de división
        self.left = left
        self.right = right

def construir_kd_tree(data, labels, depth=0):
    n = len(data)
    if n == 0:
        return None

    # Determinar eje de división (rota entre atributos)
    k = data.shape[1]
    axis = depth % k

    # Ordenar por el eje y dividir por la mediana
    sorted_indices = data[:, axis].argsort()
    data = data[sorted_indices]
    labels = labels[sorted_indices]
    med = n // 2

    return KDNode(
        point=data[med],
        label=labels[med],
        axis=axis,
        left=construir_kd_tree(data[:med], labels[:med], depth + 1),
        right=construir_kd_tree(data[med + 1:], labels[med + 1:], depth + 1)
    )


## Visualización del Árbol KD

Para interpretar la estructura del árbol KD de forma clara y comprensible, usaremos la biblioteca `pyvis` para generar una visualización interactiva en HTML. Cada nodo del árbol será representado como un círculo, con líneas que conectan a sus nodos hijo.

- **Cada nodo** mostrará:
  - El valor del punto (atributos del ejemplo).
  - El eje de división (X₀, X₁, X₂, ...).
  - La clase asignada al punto.
- **Las conexiones** representan cómo se divide recursivamente el conjunto de datos.
- El árbol puede visualizarse fácilmente como una jerarquía descendente, lo que permite identificar reglas de decisión y patrones de clasificación.

Este enfoque facilita la explicación y análisis de las divisiones que hace el árbol KD para separar las clases.


In [18]:
import pandas as pd
import numpy as np
from pyvis.network import Network
from IPython.display import IFrame

# ---------------------------
# Estructura del nodo KD
# ---------------------------
class KDNode:
    def __init__(self, point, label, axis, left=None, right=None):
        self.point = point
        self.label = label
        self.axis = axis
        self.left = left
        self.right = right

# ---------------------------
# Construcción recursiva
# ---------------------------
def construir_kd_tree(data, labels, depth=0):
    if len(data) == 0:
        return None

    k = data.shape[1]
    axis = depth % k

    sorted_indices = data[:, axis].argsort()
    data = data[sorted_indices]
    labels = labels[sorted_indices]
    med = len(data) // 2

    return KDNode(
        point=data[med],
        label=labels[med],
        axis=axis,
        left=construir_kd_tree(data[:med], labels[:med], depth + 1),
        right=construir_kd_tree(data[med + 1:], labels[med + 1:], depth + 1)
    )

# ---------------------------
# Visualización con PyVis
# ---------------------------
def agregar_nodos_aristas(net, node, parent_id=None, node_id=0, depth=0):
    if node is None:
        return node_id

    label = f"Clase: {node.label}\nEje: X{node.axis}\n{np.round(node.point, 2)}"
    current_id = node_id
    net.add_node(current_id, label=label, title=label, level=depth)

    if parent_id is not None:
        net.add_edge(parent_id, current_id)

    node_id += 1
    node_id = agregar_nodos_aristas(net, node.left, current_id, node_id, depth + 1)
    node_id = agregar_nodos_aristas(net, node.right, current_id, node_id, depth + 1)

    return node_id

def visualizar_arbol_kd(root, output_file="arbol_kd.html"):
    net = Network(height="800px", width="100%", directed=True)

    net.set_options("""
    {
      "layout": {
        "hierarchical": {
          "enabled": true,
          "levelSeparation": 120,
          "nodeSpacing": 100,
          "treeSpacing": 200,
          "direction": "UD",
          "sortMethod": "directed"
        }
      },
      "physics": {
        "hierarchicalRepulsion": {
          "nodeDistance": 120
        },
        "solver": "hierarchicalRepulsion"
      }
    }
    """)

    agregar_nodos_aristas(net, root)
    net.save_graph(output_file)
    return IFrame(output_file, width=950, height=750)

# ---------------------------
# Cargar datos y construir árbol
# ---------------------------
df = pd.read_csv("Dataset_clases.csv")
df.columns = df.columns.str.strip()

# Asignar X (atributos) y y (clase)
X = df.drop(columns=["CU Total", "COT", "Clase"], errors='ignore')  # quitar la clase y columnas dependientes
y = df["CU Total"] if "Clase" not in df.columns else df["Clase"]

# Asegurarse de que X es solo numérico
X = X.select_dtypes(include=[np.number])

# Construir y visualizar
root = construir_kd_tree(X.values, y.values)
visualizar_arbol_kd(root)
